In [ ]:
from pyknp import KNP

In [ ]:
# 定義
data_size = 215 # データサイズ
skip_file = [107,129,125,151,173,195]

file_info = "./data/input/RS_data/rsdata" # アノテーションデータのPATH
corpus_file = "corpus_5w1hs_ch5.txt" # 出力コーパスの名前

In [ ]:
def get_iobcorpus(file_name):
    
    # アノテーション情報用意
    file_ann = "{0}.ann".format(file_name)
    tag_number = [] # アノテーションラベルの先頭番号と末尾番号リスト
    tag_vocab = {} # アノテーションラベルの名前と先頭番号
    with open(file_ann, 'r') as f:
        for i in f.read().split("\n"):
            if len(i) != 0:
                t_s = int(i.split()[2]) # アノテーションの先頭
                t_e = int(i.split()[3]) # アノテーションの末尾
                tag_vocab[t_s] = i.split()[1]
                tag_number.append(t_s)
                tag_number.append(t_e)
    tag_number = sorted(tag_number)
    
    # 形態素解析
    file_txt = "{0}.txt".format(file_name)

    word_count = -1 # 単語サイズ
    n = 0 # タグ番号のインデックス
    p = -1 # アノテーションのラベルのインデックス
    features_set = [] # 記事の特徴量リスト
    
    knp = KNP()
    with open(file_txt, 'r') as f:
        data_text = f.read() # ファイル読み込み
        result = knp.parse(data_text)
        
        for bnst in result.bnst_list(): # 文節
            mrph_list = bnst.mrph_list()   
            for mrph in mrph_list: # 形態素 
                
                features = [] # 特徴
                word_count += len(mrph.midasi) # 単語サイズ計算
                features.extend([mrph.midasi, mrph.hinsi, mrph.bunrui])

                # 文末処理
                if n >= len(tag_number):
                    features.append("O")
                    features_set.append(features)
                    continue 
                    
                # タグ処理
                if n%2 == 0:
                    if tag_number[n] <= word_count:
                        p = tag_number[n]
                        features.append("B-{0}".format(tag_vocab[p]))
                        n += 1
                    else:
                        features.append("O")
                elif n%2 != 0:    
                    if tag_number[n] <= word_count:
                        features.append("O")
                        n += 1
                    else:                        
                        features.append("I-{0}".format(tag_vocab[p]))
                        
                features_set.append(features)
                
    return features_set

In [ ]:
# main
data_sum = 0

# 書き出しファイル準備
output_f = open(corpus_file, "w")

for file_id in range(1,data_size+1): 
    
    if file_id in skip_file:
        continue
        
    data_sum += 1
    print(file_id,end="")
    file_name = "{0}{1}".format(file_info,file_id)
    for line in get_iobcorpus(file_name):
        output_line = "\t".join(line)
        output_f.writelines(output_line+"\n")
    output_f.writelines("\n") # 5w1h用の改行コマンド
    
output_f.close()

print("")
print(data_sum)